In [3]:
# for importing notebooks
import nbimporter

In [4]:
from vector import Vector

Importing Jupyter notebook from vector.ipynb


In [5]:
from plane import Plane

Importing Jupyter notebook from plane.ipynb


In [6]:
from decimal import Decimal, getcontext
from copy import deepcopy

getcontext().prec = 30


class LinearSystem(object):

    ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG = 'All planes in the system should live in the same dimension'
    NO_SOLUTIONS_MSG = 'No solutions'
    INF_SOLUTIONS_MSG = 'Infinitely many solutions'

    def __init__(self, planes):
        try:
            d = planes[0].dimension
            for p in planes:
                assert p.dimension == d

            self.planes = planes
            self.dimension = d

        except AssertionError:
            raise Exception(self.ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG)


    def swap_rows(self, row1, row2):
        self[row1], self[row2] = self[row2], self [row1]


    def multiply_coefficient_and_row(self, coefficient, row):
        n = self[row].normal_vector
        c = self[row].constant_term
        self[row] = Plane(n.scalar_mul(coefficient), c*coefficient)


    def add_multiple_times_row_to_row(self, coefficient, row_to_add, row_to_be_added_to):
        n1 = self[row_to_add].normal_vector
        n2 = self[row_to_be_added_to].normal_vector
        c1 = self[row_to_add].constant_term
        c2 = self[row_to_be_added_to].constant_term
        
        new_n = n2 + Vector(n1.scalar_mul(coefficient))
        new_c = c2 + c1*coefficient
        
        self[row_to_be_added_to] = Plane(new_n, new_c)
        
        
        
#     def find_nonzero_row():
        
        
#     def clear_terms(self, variable, row):
        
        
    def compute_triangular_form(self):
        system = deepcopy(self)
        
        m = len(system)
        n = 3
        j = 0
        
#         for i in range(0,m):
#             while j < n:
#                 c = system[i].normal_vector.coordinates[j]
#                 if c == 0 :
#                     # if there is a row under row i with non-zero coeffs for var j
#                         # system.swap_rows(i, row2)
#                     # else
#                         # j = j+1
                
#                 # Clear all terms with var j below row i
#                 j = j+1
#                 break
        
        return system
        

    def indices_of_first_nonzero_terms_in_each_row(self):
        num_equations = len(self)
        num_variables = self.dimension

        indices = [-1] * num_equations

        for i,p in enumerate(self.planes):
            try:
                indices[i] = p.first_nonzero_index(p.normal_vector.coordinates)
            except Exception as e:
                if str(e) == Plane.NO_NONZERO_ELTS_FOUND_MSG:
                    continue
                else:
                    raise e

        return indices


    def __len__(self):
        return len(self.planes)


    def __getitem__(self, i):
        return self.planes[i]


    def __setitem__(self, i, x):
        try:
            assert x.dimension == self.dimension
            self.planes[i] = x

        except AssertionError:
            raise Exception(self.ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG)


    def __str__(self):
        ret = 'Linear System:\n'
        temp = ['Equation {}: {}'.format(i+1,p) for i,p in enumerate(self.planes)]
        ret += '\n'.join(temp)
        return ret


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps


In [7]:
p0 = Plane([1,1,1], 1)
p1 = Plane([0,1,0], 2)
p2 = Plane([1,1,-1], 3)
p3 = Plane([1,0,-2], 2)

s = LinearSystem([p0,p1,p2,p3])
s.swap_rows(0,1)
if not (s[0] == p1 and s[1] == p0 and s[2] == p2 and s[3] == p3):
    print('test case 1 failed')

s.swap_rows(1,3)
if not (s[0] == p1 and s[1] == p3 and s[2] == p2 and s[3] == p0):
    print('test case 2 failed')

s.swap_rows(3,1)
if not (s[0] == p1 and s[1] == p0 and s[2] == p2 and s[3] == p3):
    print('test case 3 failed')

s.multiply_coefficient_and_row(1,0)
if not (s[0] == p1 and s[1] == p0 and s[2] == p2 and s[3] == p3):
    print('test case 4 failed')

s.multiply_coefficient_and_row(-1,2)
if not (s[0] == p1 and
        s[1] == p0 and
        s[2] == Plane([-1,-1,1], -3) and
        s[3] == p3):
    print('test case 5 failed')

s.multiply_coefficient_and_row(10,1)
if not (s[0] == p1 and
        s[1] == Plane([10,10,10], 10) and
        s[2] == Plane([-1,-1,1], -3) and
        s[3] == p3):
    print('test case 6 failed')

s.add_multiple_times_row_to_row(0,0,1)
if not (s[0] == p1 and
        s[1] == Plane([10,10,10], 10) and
        s[2] == Plane([-1,-1,1], -3) and
        s[3] == p3):
    print('test case 7 failed')

s.add_multiple_times_row_to_row(1,0,1)
if not (s[0] == p1 and
        s[1] == Plane([10,11,10], 12) and
        s[2] == Plane([-1,-1,1], -3) and
        s[3] == p3):
    print('test case 8 failed')

s.add_multiple_times_row_to_row(-1,1,0)
if not (s[0] == Plane([-10,-10,-10], -10) and
        s[1] == Plane([10,11,10], 12) and
        s[2] == Plane([-1,-1,1], -3) and
        s[3] == p3):
    print('test case 9 failed')

In [8]:
print(s)

print(s.indices_of_first_nonzero_terms_in_each_row())


Linear System:
Equation 1: -10x_1 - 10x_2 - 10x_3 = -10
Equation 2: 10x_1 + 11x_2 + 10x_3 = 12
Equation 3: -x_1 - x_2 + x_3 = -3
Equation 4: x_1 - 2x_3 = 2
[0, 0, 0, 0]


In [9]:
p1 = Plane([1,1,1], 1)
p2 = Plane([0,1,1], 2)
s = LinearSystem([p1,p2])
t = s.compute_triangular_form()
print(s)
print(t)
if not (t[0] == p1 and
        t[1] == p2):
    print('test case 1 failed')
    
p1 = Plane([1,1,1], 1)
p2 = Plane([1,1,1], 2)
s = LinearSystem([p1,p2])
t = s.compute_triangular_form()
if not (t[0] == p1 and
        t[1] == Plane(constant_term=1)):
    print('test case 2 failed')

p1 = Plane([1,1,1], 1)
p2 = Plane([0,1,0], 2)
p3 = Plane([1,1,-1], 3)
p4 = Plane([1,0,-2], 2)
s = LinearSystem([p1,p2,p3,p4])
t = s.compute_triangular_form()
if not (t[0] == p1 and
        t[1] == p2 and
        t[2] == Plane([0,0,-2], 2) and
        t[3] == Plane()):
    print('test case 3 failed')

p1 = Plane([0,1,1], 1)
p2 = Plane([1,-1,1], 2)
p3 = Plane([1,2,-5], 3)
s = LinearSystem([p1,p2,p3])
t = s.compute_triangular_form()
if not (t[0] == Plane([1,-1,1], 2) and
        t[1] == Plane([0,1,1], 1) and
        t[2] == Plane([0,0,-9], -2)):
    print('test case 4 failed')

Linear System:
Equation 1: x_1 + x_2 + x_3 = 1
Equation 2: x_2 + x_3 = 2
Linear System:
Equation 1: x_1 + x_2 + x_3 = 1
Equation 2: x_2 + x_3 = 2
test case 2 failed
test case 3 failed
test case 4 failed
